In [1]:
from dotenv import load_dotenv

In [27]:
load_dotenv('/home/tom/two/envapi/my-env')

True

In [42]:
#free short stores: https://theshortstory.co.uk/resources/free-short-stories/
# use online pdf2txt to convert to txt files and download 

In [4]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain

In [8]:
do_loader = UnstructuredFileLoader("/home/tom/two/Udemy/short-stories/Oil-of-Dog-short-stories.txt")
do_doc = do_loader.load()

oc_loader = UnstructuredFileLoader("/home/tom/two/Udemy/short-stories/Owl-Creek-Bridge-shoprt-stories.txt")
oc_doc = oc_loader.load()

In [9]:
len(do_doc)

1

In [13]:
print(do_doc[0].page_content)

Oil of Dog

Ambrose Bierce

My name is Boffer Bings. I was born of honest parents in one of the

humbler walks of life, my father being a manufacturer of dog-oil and my

mother having a small studio in the shadow of the village church, where

she disposed of unwelcome babes. In my boyhood I was trained to habits

of industry; I not only assisted my father in procuring dogs for his vats,

but was frequently employed by my mother to carry away the debris of

her work in the studio. In performance of this duty I sometimes had need

of all my natural intelligence for all the law officers of the vicinity were

opposed to my mother's business. They were not elected on an opposition

ticket, and the matter had never been made a political issue; it just

happened so. My father's business of making dog-oil was, naturally, less

unpopular, though the owners of missing dogs sometimes regarded him

with suspicion, which was reflected, to some extent, upon me. My father

had, as silent partners, al

In [22]:
def doc_summary(docs):
    print(f'You have {len(docs)} documents')
    num_words = sum([len(doc.page_content.split(' ')) for doc in docs])
    print(f'There are nearly {num_words} words in the doc')
    print()
    print(f'Preview: {docs[0].page_content.split(". ")[:2]}')

In [23]:
doc_summary(do_doc)

You have 1 documents
There are nearly 1348 words in the doc

Preview: ['Oil of Dog\n\nAmbrose Bierce\n\nMy name is Boffer Bings', 'I was born of honest parents in one of the\n\nhumbler walks of life, my father being a manufacturer of dog-oil and my\n\nmother having a small studio in the shadow of the village church, where\n\nshe disposed of unwelcome babes']


In [24]:
doc_summary(oc_doc)

You have 1 documents
There are nearly 3431 words in the doc

Preview: ['An Occurrence at Owl Creek Bridge\n\nAmbrose Bierce\n\nI\n\nA man stood upon a railroad bridge in northern Alabama, looking down\n\ninto the swift water twenty feet below', "The man's hands were behind his\n\nback, the wrists bound with a cord"]


In [25]:
from langchain import OpenAI

In [29]:
llm = OpenAI(
    temperature=.9
)
chain = load_summarize_chain(
    llm=llm,
    chain_type="stuff",
    verbose=True
)
chain.run(do_doc)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Oil of Dog

Ambrose Bierce

My name is Boffer Bings. I was born of honest parents in one of the

humbler walks of life, my father being a manufacturer of dog-oil and my

mother having a small studio in the shadow of the village church, where

she disposed of unwelcome babes. In my boyhood I was trained to habits

of industry; I not only assisted my father in procuring dogs for his vats,

but was frequently employed by my mother to carry away the debris of

her work in the studio. In performance of this duty I sometimes had need

of all my natural intelligence for all the law officers of the vicinity were

opposed to my mother's business. They were not elected on an opposition

ticket, and the matter had never been made a political issue; it just

happened so. My father's business of making dog-oil was, naturally, less

unpopular, though 


> Finished chain.

> Finished chain.


" Boffer Bings is born to parents in the humbler walks of life, his father a dog-oil manufacturer and his mother disposing of unwelcome babes in her studio. Boffer helps his father procure dogs for his vats, and his mother with disposing of unwanted babes. Boffer accidentally casts a foundling into his father's cauldron, and the next day his father notices the high quality of the oil. His parents begin to gather unwelcomed children and adults to obtain higher profits. They are censured for their actions and in a fit of despair they both jump into the same cauldron, dying in the process. Boffer, feeling responsible, moves to the city Otumwee where he is writing these memoirs."

In [30]:
chain.run(oc_doc)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"An Occurrence at Owl Creek Bridge

Ambrose Bierce

I

A man stood upon a railroad bridge in northern Alabama, looking down

into the swift water twenty feet below. The man's hands were behind his

back, the wrists bound with a cord. A rope closely encircled his neck. It

was attached to a stout cross-timber above his head and the slack fell to

the level of his knees. Some loose boards laid upon the sleepers

supporting the metals of the railway supplied a footing for him and his

executioners--two private soldiers of the Federal army, directed by a

sergeant who in civil life may have been a deputy sheriff. At a short

remove upon the same temporary platform was an officer in the uniform

of his rank, armed. He was a captain. A sentinel at each end of the bridge

stood with his rifle in the position known as "support," that is to say,



InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 5633 tokens (5377 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

In [31]:
#switch to chain type 'map_reduce'
chain = load_summarize_chain(
    llm = llm, 
    chain_type="map_reduce",
    verbose=True
)

In [34]:
#chain.run(oc_doc) - same issue, so we need to split large doc to smaller overlapping chunks

In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [36]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400, 
    chunk_overlap=50
)

In [37]:
oc_docs_chunks = text_splitter.split_documents(oc_doc)

In [39]:
oc_docs_chunks[:3]

[Document(page_content="An Occurrence at Owl Creek Bridge\n\nAmbrose Bierce\n\nI\n\nA man stood upon a railroad bridge in northern Alabama, looking down\n\ninto the swift water twenty feet below. The man's hands were behind his\n\nback, the wrists bound with a cord. A rope closely encircled his neck. It\n\nwas attached to a stout cross-timber above his head and the slack fell to", metadata={'source': '/home/tom/two/Udemy/short-stories/Owl-Creek-Bridge-shoprt-stories.txt'}),
 Document(page_content='the level of his knees. Some loose boards laid upon the sleepers\n\nsupporting the metals of the railway supplied a footing for him and his\n\nexecutioners--two private soldiers of the Federal army, directed by a\n\nsergeant who in civil life may have been a deputy sheriff. At a short\n\nremove upon the same temporary platform was an officer in the uniform', metadata={'source': '/home/tom/two/Udemy/short-stories/Owl-Creek-Bridge-shoprt-stories.txt'}),
 Document(page_content='of his rank, arme

In [40]:
#chain.run(oc_docs_chunks)
doc_summary(oc_docs_chunks)

You have 60 documents
There are nearly 3507 words in the doc

Preview: ['An Occurrence at Owl Creek Bridge\n\nAmbrose Bierce\n\nI\n\nA man stood upon a railroad bridge in northern Alabama, looking down\n\ninto the swift water twenty feet below', "The man's hands were behind his\n\nback, the wrists bound with a cord"]


In [41]:
chain.run(oc_docs_chunks[:5])



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"An Occurrence at Owl Creek Bridge

Ambrose Bierce

I

A man stood upon a railroad bridge in northern Alabama, looking down

into the swift water twenty feet below. The man's hands were behind his

back, the wrists bound with a cord. A rope closely encircled his neck. It

was attached to a stout cross-timber above his head and the slack fell to"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"the level of his knees. Some loose boards laid upon the sleepers

supporting the metals of the railway supplied a footing for him and his

executioners--two private soldiers of the Federal army, directed by a

sergeant who in civil life may have been a deputy sheriff. At a short

remove upon the same temporary platform was an officer in the uniform"


CONCISE SUMMARY:
Prompt after formatting:
Write a conci

' \nA man is bound with rope on a railroad bridge in Alabama. He is watched by two captains with rifles, an officer in uniform, two private soldiers, and a sergeant. An infantry company is lined up to observe. The circumstances of the situation are unknown, but there is a fort with a cannon overlooking the scene.'

In [43]:
#Summarize using Refine

In [44]:
chain = load_summarize_chain(
    llm=llm, 
    chain_type="refine",
    verbose=True
)

In [45]:
chain.run(oc_docs_chunks[:5])



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"An Occurrence at Owl Creek Bridge

Ambrose Bierce

I

A man stood upon a railroad bridge in northern Alabama, looking down

into the swift water twenty feet below. The man's hands were behind his

back, the wrists bound with a cord. A rope closely encircled his neck. It

was attached to a stout cross-timber above his head and the slack fell to"


CONCISE SUMMARY:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: 

A man stands upon a railroad bridge in northern Alabama with his hands behind his back, bound by a cord and a rope around his neck attached to a cross-timber above him. He is looking down into the swift water twenty feet below.
We have the opportunity to refine the existing summary (only if needed) w

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-pVOnO8dpEcNyuE7hDq8Ei5vA on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-pVOnO8dpEcNyuE7hDq8Ei5vA on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.l


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: 

A man stands upon a railroad bridge in northern Alabama with his hands behind his back, bound by a cord and a rope around his neck attached to a cross-timber above him. He is looking down into the swift water twenty feet below, supported by two private soldiers of the Federal army and a sergeant who may have been a deputy sheriff in civilian life. An officer in uniform, of his rank, armed and identified as a captain stands at a short distance on the same platform, with a sentinel at each end of the bridge, standing with their rifles in the position of "support". The sentinels were tasked with blocking the foot planking that traversed the bridge, while an outpost was located at the further end beyond one of them, where the railroad ran straight away into a forest.
We have the opportunity to refine the existin

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-pVOnO8dpEcNyuE7hDq8Ei5vA on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-pVOnO8dpEcNyuE7hDq8Ei5vA on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.l


> Finished chain.

> Finished chain.


'\n\nA man stands upon a railroad bridge in northern Alabama with his hands behind his back, bound by a cord and a rope around his neck attached to a cross-timber above him. He is looking down into the swift water twenty feet below, supported by two private soldiers of the Federal army and a sergeant who may have been a deputy sheriff in civilian life. An officer in uniform, of his rank, armed and identified as a captain stands at a short distance on the same platform, with a sentinel at each end of the bridge, standing with their rifles in the position of "support". The sentinels were tasked with blocking the foot planking that traversed the bridge, while an outpost was located at the further end beyond one of them, where the railroad ran straight away into a forest. Midway of the slope between the bridge and a fort made of vertical tree trunks, loopholed for rifles and commanded by a brass cannon, stood a single company of infantry in line at "parade rest".'

In [46]:
#Q&A - Map re-rank

In [48]:
chain = load_qa_chain(
    llm=llm, 
    chain_type="map_rerank",
    verbose=True,
    return_intermediate_steps=True
)

In [49]:
query = "How long did the railroad run into the forest before curving?"
result = chain( 
    {"input_documents":oc_docs_chunks[:5], "question":query },
    return_only_outputs= True
)



> Entering new MapRerankDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer: [answer here]
Score: [score between 0 and 100]

How to determine the score:
- Higher is a better answer
- Better responds fully to the asked question, with sufficient level of detail
- If you do not know the answer based on the context, that should be a score of 0
- Don't be overconfident!

Example #1

Context:
---------
Apples are red
---------
Question: what color are apples?
Helpful Answer: red
Score: 100

Example #2

Context:
---------
it was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv
--

/home/tom/.local/lib/python3.10/site-packages/langchain/chains/llm.py:349: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.

> Finished chain.


In [50]:
result['output_text']

' Around 100 yards.'

In [51]:
result['intermediate_steps']

[{'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' Around 100 yards.', 'score': '100'},
 {'answer': ' This document does not answer the question.', 'score': '0'}]